### Importing libraries

In [ ]:
import pandas as pd
import optuna
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor


/home/alexander/Documents/GitHub/OGC-2025---Canopy-monitoring-hackathon/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading training data

The training dataset is loaded using [pandas](https://pandas.pydata.org/). After separating the predictors (X) from the variable to be estimated (y) the dataset is split into a dataset for training and testing the point estimators as well as a calibration dataset to conformalise the qunatile regressors used to estimate predictive uncertainty. While the training/testing split is done with a 80/20 ratio the testing/calibration split is conducted using a 50/50 ratio. The splitting is facilitated using the train_test_split functionality of [https://scikit-learn.org](scikit-learn).

In [2]:
training_data = pd.read_csv("./data/train.csv").drop(columns=["lat", "lon"])
variables = list(training_data.columns)

y = training_data["rh98"]
X = training_data.drop(columns=["rh98"])
    

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_calib, y_train, y_calib = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

## Model tuning and fitting
Before fiting the models, in this case two tree-base ensemble approaches namely [Light Gradient Boosting](https://lightgbm.readthedocs.io/en/stable/) and [Extreme Gradient Boosting](https://xgboost.readthedocs.io/en/stable/), their hyperparameters are tuned on the training dataset using thr optimisation framework [Optuna](https://optuna.readthedocs.io/en/stable/). This is done yb defining an objective function alongside the parameter space. The Roost Mean Squared Error is used as the score function which has to be minimize. The tuhing ran for 300 epochs. Optuna also provides a dashboard to reviev tuning performance and assess the importance of individual hyperparameters. In can be started using the following command: optuna-dashboard sqlite:///tuning.sqlite3. I can be accessed through: [http://127.0.0.1:8080/dashboard/](http://127.0.0.1:8080/dashboard/). The tuning results are stored in a sqlite database for later use or extension. 

#### Tuning the Light Gradient Boosting Regressor 

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.5)
    max_depth = trial.suggest_int("max_depth", 3, 50)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0001, 100, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0001, 100, log=True)

    model = LGBMRegressor(objective='regression',
                                   n_estimators=n_estimators,
                                   learning_rate=learning_rate,
                                   max_depth=max_depth,
                                   colsample_bytree=colsample_bytree,
                                   subsample=subsample,
                                   boosting_type='gbdt',
                                   reg_alpha=reg_alpha,
                                   reg_lambda=reg_lambda,
                                   random_state=42,
                                   n_jobs=12,
                                   verbosity=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = root_mean_squared_error(y_test, y_pred)
    
    return score

study = optuna.create_study(study_name=f"OGH_2025_Canopy_Prediction_LightGradientBoosting", direction="minimize", storage="sqlite:///tuning.sqlite3", load_if_exists=False)
study.optimize(objective, n_trials=300)
print(f"Best parameters: {study.best_params}")

[I 2025-09-01 18:43:58,362] A new study created in RDB with name: OGH_2025_Canopy_Prediction_LightGradientBoosting
[I 2025-09-01 18:43:59,452] Trial 0 finished with value: 545.5252802240565 and parameters: {'n_estimators': 230, 'learning_rate': 0.11098119809884484, 'max_depth': 33, 'colsample_bytree': 0.7904109263732374, 'subsample': 0.17800049170036125, 'reg_alpha': 49.77915496614197, 'reg_lambda': 0.03463935334657587}. Best is trial 0 with value: 545.5252802240565.
[I 2025-09-01 18:44:02,256] Trial 1 finished with value: 577.7321065484967 and parameters: {'n_estimators': 738, 'learning_rate': 0.26098637900205124, 'max_depth': 25, 'colsample_bytree': 0.46370729971116564, 'subsample': 0.5651438985785908, 'reg_alpha': 0.002116302887421061, 'reg_lambda': 0.03601919519433039}. Best is trial 0 with value: 545.5252802240565.
[I 2025-09-01 18:44:04,073] Trial 2 finished with value: 544.2039566197283 and parameters: {'n_estimators': 424, 'learning_rate': 0.09679412220424606, 'max_depth': 30, 

Best parameters: {'n_estimators': 949, 'learning_rate': 0.020337307107457278, 'max_depth': 10, 'colsample_bytree': 0.30972941585327557, 'subsample': 0.14096608560952045, 'reg_alpha': 0.0020958178503886157, 'reg_lambda': 99.94738437044839}


#### Tuning the Extreme Gradient Boosting Regressor 

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.5)
    max_depth = trial.suggest_int("max_depth", 3, 50)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    eval_metric = trial.suggest_categorical("eval_metric", ['rmse', 'mae', 'logloss'])
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0001, 100, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0001, 100, log=True)

    model = XGBRegressor(n_estimators=n_estimators, 
                         max_depth=max_depth, 
                         subsample=subsample, 
                         colsample_bytree=colsample_bytree, 
                         eval_metric=eval_metric,
                         learning_rate=learning_rate,
                         booster='gbtree', 
                         reg_alpha=reg_alpha,
                         reg_lambda=reg_lambda,
                         n_jobs=12, 
                         seed=42)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = root_mean_squared_error(y_test, y_pred)
    
    return score

study = optuna.create_study(study_name=f"OGH_2025_Canopy_Prediction_ExtremeGradientBoosting", direction="minimize", storage="sqlite:///tuning.sqlite3", load_if_exists=False)
study.optimize(objective, n_trials=300)
print(f"Best parameters: {study.best_params}")

#optuna-dashboard sqlite:///tuning.sqlite3

[I 2025-09-01 18:55:05,396] A new study created in RDB with name: OGH_2025_Canopy_Prediction_ExtremeGradientBoosting
[I 2025-09-01 18:55:06,220] Trial 0 finished with value: 589.63916015625 and parameters: {'n_estimators': 999, 'learning_rate': 0.3486474672741869, 'max_depth': 3, 'subsample': 0.5955941506173542, 'eval_metric': 'logloss', 'colsample_bytree': 0.8685192401860561, 'reg_alpha': 0.0011877947879153892, 'reg_lambda': 23.577963275111266}. Best is trial 0 with value: 589.63916015625.
[I 2025-09-01 18:55:08,986] Trial 1 finished with value: 615.1828002929688 and parameters: {'n_estimators': 55, 'learning_rate': 0.34439974331171574, 'max_depth': 42, 'subsample': 0.7846418209113181, 'eval_metric': 'logloss', 'colsample_bytree': 0.730314036648714, 'reg_alpha': 37.15374580128859, 'reg_lambda': 0.021146693712777383}. Best is trial 0 with value: 589.63916015625.
[I 2025-09-01 18:55:16,269] Trial 2 finished with value: 648.0765380859375 and parameters: {'n_estimators': 644, 'learning_ra

Best parameters: {'n_estimators': 965, 'learning_rate': 0.022467560517086513, 'max_depth': 7, 'subsample': 0.4379916528711094, 'eval_metric': 'mae', 'colsample_bytree': 0.879809082567292, 'reg_alpha': 0.05877334178011133, 'reg_lambda': 97.29817113759651}


## Fitting the regressors

The two tuned regressors are now fitted to the training dataset using the best performing hyperparameters from the studies stored in the sqlite database. The trained models are subsequently pickled for later use. 
The regressors achived the following performance on the test dataset:

|Model                    | Coefficient of determination|Root Mean Squared Error|
|-------------------------|-----------------------------|-----------------------|
|Light Gradient Boosting  |0.6657                       |535.31                 |
|Extreme Gradient Boosting|0.6669                       |534.32                 |

Since the Extreme Gradient Boosting performed slightly better than the Light Gradient Boosting the former one is used for the remainder of the study.


In [10]:
study_lgb = optuna.load_study(study_name=f"OGH_2025_Canopy_Prediction_LightGradientBoosting", storage="sqlite:///tuning.sqlite3")
lgb = LGBMRegressor(**study_lgb.best_params, random_state=42)
lgb.fit(X_train, y_train)
with open(f'./lightGradientBoosting.pickle', 'wb') as handle:
        pickle.dump(lgb, handle, protocol=pickle.HIGHEST_PROTOCOL)

y_pred = lgb.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rsme = root_mean_squared_error(y_test, y_pred)
print(f'LGB: Mean squared error: {mse} R2 score: {r2} Mean absolute error: {mae} Root mean squared error: {rsme}')

study_xgb = optuna.load_study(study_name=f"OGH_2025_Canopy_Prediction_ExtremeGradientBoosting", storage="sqlite:///tuning.sqlite3")
xgb = XGBRegressor(**study_xgb.best_params, random_state=42)
xgb.fit(X_train, y_train)
with open('./extremeGradientBoosting.pickle', 'wb') as handle:
        pickle.dump(xgb, handle, protocol=pickle.HIGHEST_PROTOCOL)

y_pred = xgb.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rsme = root_mean_squared_error(y_test, y_pred)
print(f'XGB: Mean squared error: {mse} R2 score: {r2} Mean absolute error: {mae} Root mean squared error: {rsme}')

LGB: Mean squared error: 286566.36332897225 R2 score: 0.6657552970229692 Mean absolute error: 385.88599419262897 Root mean squared error: 535.3189360829414
XGB: Mean squared error: 285501.15625 R2 score: 0.6669977903366089 Mean absolute error: 384.5604553222656 Root mean squared error: 534.3230590820312


## Uncertainty quantification

Point estimates alongside the performance indicators of the underlying estimator might be lacking utility and interpretability in certain use cases. Infromaton regading the uncertainly and reliability of the point estimates might improve on both aspects. Quantifying uncertainties can be a daunting task especially in complex modeling scenarios. A straightforward way of qunatifying uncertainties is to fit additional qunatile regressors corresponding to the desired confidence level, here 95%.

Two Light Gradient Boosting regressors are fitted to the 0.025 and 0.975 quantile respectively. 

In [ ]:
confidence = 0.95
miscoverage = 1-confidence
alpha_low = round(miscoverage/2, 3)
alpha_upp = round(1-miscoverage/2, 3)

def objective(trial, alpha):
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.5)
    max_depth = trial.suggest_int("max_depth", 3, 50)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0001, 100, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0001, 100, log=True)
    
    model = LGBMRegressor(objective='quantile',
                          alpha=alpha,
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth,
                        colsample_bytree=colsample_bytree,
                        subsample=subsample,
                        boosting_type='gbdt',
                        reg_alpha=reg_alpha,
                        reg_lambda=reg_lambda,
                        verbose=-1,
                        random_state=42,
                        n_jobs=12)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_train) 
    score = root_mean_squared_error(y_train, y_pred)
    
    return score

study_low = optuna.create_study(study_name=f"OGH_2025_Canopy_Prediction_{alpha_low}_QuantileRegressor", direction="minimize", storage="sqlite:///tuning.sqlite3")
study_low.optimize(lambda trial: objective(trial, alpha=alpha_low), n_trials=300)


study_upp = optuna.create_study(study_name=f"OGH_2025_Canopy_Prediction_{alpha_upp}_QuantileRegressor", direction="minimize", storage="sqlite:///tuning.sqlite3")
study_upp.optimize(lambda trial: objective(trial, alpha=alpha_upp), n_trials=300)



[I 2025-09-01 23:27:32,424] A new study created in RDB with name: OGH_2025_Canopy_Prediction_0.025_QuantileRegressor
[I 2025-09-01 23:27:33,159] Trial 0 finished with value: 1449.1276655667202 and parameters: {'n_estimators': 149, 'learning_rate': 0.016377365733248602, 'max_depth': 32, 'colsample_bytree': 0.30262317497850655, 'subsample': 0.31043891610518876, 'reg_alpha': 0.00016519803027374144, 'reg_lambda': 1.4432150300653923}. Best is trial 0 with value: 1449.1276655667202.
[I 2025-09-01 23:27:35,774] Trial 1 finished with value: 986.0893871042142 and parameters: {'n_estimators': 647, 'learning_rate': 0.4481609275436009, 'max_depth': 31, 'colsample_bytree': 0.3572450393715214, 'subsample': 0.21912328047402524, 'reg_alpha': 0.008757767974516897, 'reg_lambda': 0.00489220314468725}. Best is trial 1 with value: 986.0893871042142.
[I 2025-09-01 23:27:38,406] Trial 2 finished with value: 1007.0836722584027 and parameters: {'n_estimators': 875, 'learning_rate': 0.35480524450010736, 'max_de

In [11]:
lgb_low = optuna.load_study(study_name=f"OGH_2025_Canopy_Prediction_{alpha_low}_QuantileRegressor", storage="sqlite:///tuning.sqlite3")
lgb_low = LGBMRegressor(**study_low.best_params, random_state=42)
lgb_low.fit(X_train, y_train)
with open(f'./{alpha_low}_quantileRegressor.pickle', 'wb') as handle:
        pickle.dump(lgb_low, handle, protocol=pickle.HIGHEST_PROTOCOL)

lgb_upp = optuna.load_study(study_name=f"OGH_2025_Canopy_Prediction_{alpha_upp}_QuantileRegressor", storage="sqlite:///tuning.sqlite3")
lgb_upp = LGBMRegressor(**study_upp.best_params, random_state=42)
lgb_upp.fit(X_train, y_train)
with open(f'./{alpha_upp}_quantileRegressor.pickle', 'wb') as handle:
        pickle.dump(lgb_upp, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Calibration of quantile regressors

The efficiency and reliability of the uncertainty estimates can assessed using the Size and the Coverage metrcis. The Size captures the mean length of the prediction intervals while the Coverage refers to the proportion of true values that fall within the estimated intervals. The former should be as smal as possible (depending on the use case) while the latter should at east match the desired confidence / miscoverage level [2, 3, 4].

## Literature

[1] C. Bonannella, Y.-F. Ho. Canopy height mapping using Google 'AEF embeddings'. [https://kaggle.com/competitions/canopy-height-mapping-using-google-aef-embeddings](https://kaggle.com/competitions/canopy-height-mapping-using-google-aef-embeddings), 2025. Kaggle.

[2] J. Lei, M. G’Sell, A. Rinaldo, R. J. Tibshirani, and L. Wasserman, ‘Distribution-Free Predictive Inference For Regression’, Journal of the American Statistical Association, vol. 113, Aug. 2018, doi: 10.1080/01621459.2017.1307116.

[3] H. Linusson, U. Johansson, and T. Löfström, ‘Signed-Error Conformal Regression’, in Proceedings of the 18th Pacific-Asia Conference, Tainan, Taiwan, May 2014. doi: 10.1007/978-3-319-06608-0.

[4] Y. Romano, E. Patterson, and E. J. Candès, ‘Conformalized quantile regression’, in Proceedings of the 33rd International Conference on Neural Information Processing Systems, Vancouver, Canada Aug. 2019. doi: 10.48550/arXiv.1905.03222.
